In [70]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D


In [71]:
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")
test_survived = pd.read_csv("/content/gender_submission.csv")

train_data.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


In [72]:

train_data = train_data.replace(['female','male'],[0,1])
train_data = train_data.replace(['S','C','Q'],[0,1,2])
train_data.fillna(0, inplace=True)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,0.0


In [73]:
x_train = train_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y_train = train_data[['Survived']]
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape , y_train.shape)

(891, 7) (891, 1)


In [74]:
test_data = test_data.replace(['female', 'male'],[0,1])
test_data = test_data.replace(['S','C','Q'],[0,1,2])
test_data.fillna(0, inplace=True)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,0,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,0,0
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,0,2
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,0,0


In [75]:
x_test = test_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y_test = test_survived[['Survived']]
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_test.shape , y_test.shape)

(418, 7) (418, 1)


In [76]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(12).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(12).batch(16)

In [77]:
class model_titanic(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.d1 = Dense(16, activation='relu')
    self.d2 = Dense(128, activation='relu')
    self.d3 = Dense(2, activation='softmax')

  def call(self,x):
    x=self.d1(x)
    x=self.d2(x)
    x=self.d3(x)
    return x

model = model_titanic()


In [78]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()


In [79]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [80]:
@tf.function
def train(x_train, y_train):
  with tf.GradientTape() as tape:
    pred = model(x_train)
    loss= loss_function(y_train, pred)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(y_train, pred)

In [81]:
@tf.function
def test(x_test, y_test):
  pred = model(x_test)
  loss = loss_function(y_test, pred)
  test_loss(loss)
  test_accuracy(y_test, pred)

In [82]:
epochs = 35

for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  # train
  for x_train, y_train in train_dataset:
    train(x_train, y_train)

  # test
  for x_test, y_test in test_dataset:
    test(x_test, y_test)


    print("epoch: ", epoch+1,
          f"train_loss: ,{train_loss.result()}",
          f"train_accuracy: ,{train_accuracy.result()}",
          f"test_loss: ,{test_loss.result()}",
          f"test_accuracy: ,{test_accuracy.result()}"

          )


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


epoch:  1 train_loss: ,0.6730896830558777 train_accuracy: ,0.6711559891700745 test_loss: ,0.6280055046081543 test_accuracy: ,0.75
epoch:  1 train_loss: ,0.6730896830558777 train_accuracy: ,0.6711559891700745 test_loss: ,0.7751830816268921 test_accuracy: ,0.59375
epoch:  1 train_loss: ,0.6730896830558777 train_accuracy: ,0.6711559891700745 test_loss: ,0.726243793964386 test_accuracy: ,0.5625
epoch:  1 train_loss: ,0.6730896830558777 train_accuracy: ,0.6711559891700745 test_loss: ,0.6866958141326904 test_accuracy: ,0.609375
epoch:  1 train_loss: ,0.6730896830558777 train_accuracy: ,0.6711559891700745 test_loss: ,0.7683825492858887 test_accuracy: ,0.5874999761581421
epoch:  1 train_loss: ,0.6730896830558777 train_accuracy: ,0.6711559891700745 test_loss: ,0.7702897191047668 test_accuracy: ,0.5729166865348816
epoch:  1 train_loss: ,0.6730896830558777 train_accuracy: ,0.6711559891700745 test_loss: ,0.7540324926376343 test_accuracy: ,0.5892857313156128
epoch:  1 train_loss: ,0.673089683055877

In [83]:
model.save("Titanic_model")

In [84]:
model = tf.keras.models.load_model("Titanic_model")